In [2]:
import datetime
import random
import string
from difflib import get_close_matches
import requests # For fetching external data like weather

# Response dictionary - multiple replies for natural feel
responses = {
    "hi": ["Hello! How can I help you today?", "Hi there! Nice to meet you."],
    "hello": ["Hello! How can I help you today?", "Hey! What's up?"],
    "hey": ["Hey! How's it going?", "Hello there!"],
    "what is your name": ["I'm your AI assistant, ready to help with anything!"],
    "whats your name": ["I'm your AI assistant, ready to help with anything!"],
    "who are you": ["I'm your AI assistant, ready to help with anything!"],
    "how are you": ["I'm doing great, thank you! How about you?", "Fantastic! Thanks for asking."],
    "bye": ["Goodbye! Have a great day! 👋", "See you later! Take care."],
    "goodbye": ["Goodbye! Have a great day! 👋", "Bye for now!"],
    "thank you": ["You're very welcome!", "My pleasure!"],
    "thanks": ["You're very welcome!", "No problem at all!"],
    "tell me a joke": [
        "Why don't scientists trust atoms? Because they make up everything! 😂",
        "Why did the scarecrow win an award? Because he was outstanding in his field!"
    ],
    "how old are you": ["I'm ageless – I'm an AI!", "I don't age, but I was created in 2023!"],
    "what time is it": ["The current time is {time}."],
    "what is the date today": ["Today's date is {date}."],
    "how is the weather": ["I'm sorry, I cannot provide real-time weather information at the moment. However, for Karachi, the weather is often sunny and warm!"],
    "what is the weather like in karachi": ["Let me check... Ah, I cannot fetch live weather data right now, but historically, Karachi usually has warm and humid weather with sunny skies!"],
    "default": ["I'm not sure how to respond to that. Can you rephrase?", "That's an interesting thought! Could you tell me more?", "I'm still learning! Could you try asking something else?"]
}

def clean_input(text):
    """Clean user input: lowercase, remove punctuation, strip spaces"""
    text = text.lower().strip()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def get_greeting():
    """Generates a time-based greeting."""
    current_hour = datetime.datetime.now().hour
    if 5 <= current_hour < 12:
        return "Good morning!"
    elif 12 <= current_hour < 18:
        return "Good afternoon!"
    else:
        return "Good evening!"

def fuzzy_match(user_input, response_keys, threshold=0.6):
    """Finds the best matching response category using fuzzy matching."""
    matches = get_close_matches(user_input, response_keys, n=1, cutoff=threshold)
    return matches[0] if matches else None

def get_chatbot_response(user_input):
    """Generates a response based on user input, including dynamic features."""
    cleaned_input = clean_input(user_input)

    # Check for exit commands first
    if cleaned_input in ["exit", "quit", "bye", "goodbye"]:
        return random.choice(responses["bye"])

    # Dynamic features
    if "time" in cleaned_input:
        current_time = datetime.datetime.now().strftime("%I:%M %p")
        return random.choice(responses["what time is it"]).format(time=current_time)
    elif "date" in cleaned_input:
        current_date = datetime.datetime.now().strftime("%A, %B %d, %Y")
        return random.choice(responses["what is the date today"]).format(date=current_date)
    elif "weather" in cleaned_input and "karachi" in cleaned_input:
        # Placeholder for actual weather API call
        # In a real scenario, you'd use a library like 'requests' here
        # Example: try:
        #              response = requests.get(WEATHER_API_URL)
        #              data = response.json()
        #              weather = data['weather'][0]['description']
        #              temp = data['main']['temp']
        #              return f"The weather in Karachi is {weather} with a temperature of {temp}°C."
        #          except Exception: pass
        return random.choice(responses["what is the weather like in karachi"])
    elif "weather" in cleaned_input:
        return random.choice(responses["how is the weather"])

    # Fuzzy match for general responses
    matched_key = fuzzy_match(cleaned_input, responses.keys())

    if matched_key:
        return random.choice(responses[matched_key])
    else:
        return random.choice(responses["default"])

def log_conversation(log_file, user_message, bot_message):
    """Logs the conversation to a file with timestamps."""
    timestamp = datetime.datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
    with open(log_file, "a", encoding="utf-8") as f:
        f.write(f"{timestamp} User: {user_message}\n")
        f.write(f"{timestamp} Bot: {bot_message}\n")

def session_summary(conversation_history):
    """Generates a summary of the chat session."""
    total_messages = len(conversation_history) // 2 # User and bot messages
    if total_messages == 0:
        return "No conversation took place during this session."

    user_questions = [msg for i, msg in enumerate(conversation_history) if i % 2 == 0]
    bot_responses = [msg for i, msg in enumerate(conversation_history) if i % 2 != 0]

    unique_topics = set()
    for msg in user_questions:
        words = clean_input(msg).split()
        # Simple topic extraction (can be improved with NLU)
        if "name" in words: unique_topics.add("identity")
        if "joke" in words: unique_topics.add("humor")
        if "time" in words or "date" in words: unique_topics.add("time/date")
        if "weather" in words: unique_topics.add("weather")
        if any(greet in words for greet in ["hi", "hello", "hey"]): unique_topics.add("greeting")
        if any(farewell in words for farewell in ["bye", "goodbye"]): unique_topics.add("farewell")
        if any(thanks in words for thanks in ["thank", "thanks"]): unique_topics.add("gratitude")

    summary = f"\n--- Chat Session Summary ---\n"
    summary += f"Total messages exchanged: {total_messages} (User: {len(user_questions)}, Bot: {len(bot_responses)})\n"
    summary += f"Topics discussed: {', '.join(unique_topics) if unique_topics else 'General'}\n"
    summary += "Thank you for chatting!\n--------------------------\n"
    return summary

def main_chat_loop():
    """Main loop for the chatbot interaction."""
    log_file = "chatbot_conversation.log"
    conversation_history = []
    print("\n=========================================")
    print("  Welcome to the Professional Chatbot!")
    print("=========================================")
    print(f"{get_greeting()} Type 'exit' or 'quit' to end the chat.")

    while True:
        try:
            user_input = input("You: ")
            conversation_history.append(user_input)

            if clean_input(user_input) in ["exit", "quit", "bye", "goodbye"]:
                bot_response = random.choice(responses["bye"])
                print(f"Bot: {bot_response}")
                log_conversation(log_file, user_input, bot_response)
                break

            bot_response = get_chatbot_response(user_input)
            print(f"Bot: {bot_response}")
            conversation_history.append(bot_response)
            log_conversation(log_file, user_input, bot_response)

        except EOFError:
            print("\nUser exited unexpectedly.")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            bot_response = random.choice(responses["default"])
            print(f"Bot: {bot_response}")
            log_conversation(log_file, user_input if 'user_input' in locals() else "ERROR_NO_INPUT", bot_response)

    print(session_summary(conversation_history))

# Run the chatbot
if __name__ == "__main__":
    main_chat_loop()



  Welcome to the Professional Chatbot!
Good morning! Type 'exit' or 'quit' to end the chat.
You: HI
Bot: Hello! How can I help you today?
You: How are you/
Bot: Fantastic! Thanks for asking.
You: What is your name?
Bot: I'm your AI assistant, ready to help with anything!
You: What is your age?
Bot: I'm your AI assistant, ready to help with anything!
You: tell me a joke.
Bot: Why don't scientists trust atoms? Because they make up everything! 😂
You: by
Bot: See you later! Take care.
You: 'quit'
Bot: Goodbye! Have a great day! 👋

--- Chat Session Summary ---
Total messages exchanged: 6 (User: 7, Bot: 6)
Topics discussed: humor, identity, greeting
Thank you for chatting!
--------------------------

